In [1]:
import pandas as pd 
import requests
import json
import time
import urllib.request
import glob
from bs4 import BeautifulSoup

In [2]:
'''
##############################################################################
This function will return New York Times urls from 2019 for the given keyword.
##############################################################################
'''
def nyt_url_list(keyword):
    
    #Get around 125-150 nyt urls for the given keyword from 2019
    url_list = [] #stores urls as a list
    for pagenum in range(1, 15):
        time.sleep(10)  #Sleep to avoid hitting the per minute rate
        url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?q="+str(keyword)+"&begin_date=20190101&page="+str(pagenum)+"&api-key=flkAlABqreineiWdn9AJjGRuu4QAsknD"
        resp = requests.get(url)
        data = json.loads(resp.text)
        for item in data["response"]["docs"]:
            url_list.append(item["web_url"])
            
    return url_list        

In [3]:
'''
############################################################################################
This function will take New York Times urls as input, 
extracts paragraphs from each url content and dumps it in to a created text file
############################################################################################
'''
def getArticlesText(url_list, filename):
    
    text_list = [] #stores articles text
    for url in url_list:
        try:
            r = urllib.request.urlopen(url).read()
        except:
            pass
        else:
            soup = BeautifulSoup(r, 'lxml')
            paras = soup.findAll('p')
            text = " "
            for tag in paras:
                text = text + tag.getText()
            text_list.append(text)
            
    #Create a text file of given name and store the articles text in that file.
    df = pd.DataFrame(text_list)
    df.to_csv('../Data/NYT/'+str(filename)+'.txt', index=False)

In [ ]:
sub_topics = ["Jeff+Bezos", "Mark+Zuckerberg", "Elon+Musk", "Pichai", "Tim+Cook"]

for keyword in sub_topics:
    filename = "NYT_"+ str(keyword.replace('+', ''))
    urls = nyt_url_list(keyword)
    getArticlesText(urls, filename)
    time.sleep(60) #sleep for a minute before collecting the next topic articles  

### Combine all sub-topics articles into a single text file which is passed as input to MR.

In [5]:
read_files = glob.glob('../Data/NYT/*.txt')
combined_file = '../Data/NYT\\NYT_articles.txt'

if combined_file in read_files:
    read_files.remove(combined_file)
    
with open('../Data/NYT/NYT_articles.txt', 'wb') as out:
    for file in read_files:
        with open(file, 'rb') as f:
            out.write(f.read())